## Data loading

In [5]:
# Data format:
#
# n0, n1, ...
# mu0, k00, k01, ..., k0n
# ...
# mun, kn0, kn1, ..., knn

import csv
import numpy as np

#data_file_name = 'data/axioms-toy.csv'
#data_file_name = 'data/data-tettamanzi-little.csv'
data_file_name = 'data/data-tettamanzi-complete.csv'

with open(data_file_name) as data_file:
    data = np.array(list(csv.reader(data_file)))

n = len(data) - 1

print '%d data items' % n

1444 data items


## Extract data names, membership values and Gram matrix

In [6]:
n = 1444

names = np.array(data[0])[1:n+1]
mu = np.array([float(row[0]) for row in data[1:n+1]])
gram = np.array([[float(k.replace('NA', '0')) for k in row[1:n+1]] for row in data[1:n+1]])

assert(len(names.shape) == 1)
assert(len(mu.shape) == 1)
assert(len(gram.shape) == 2)
assert(names.shape[0] == gram.shape[0] == gram.shape[1] == mu.shape[0])

## Compute adjustement in case of ill-conditioned Gram matrix

In [7]:
eigvals = np.linalg.eigvals(gram)
assert(sum([abs(e.imag) for e in eigvals]) < 1e-4)
abs_neg_eigvals = [-l.real for l in eigvals if l < 0]
adjustment = max(abs_neg_eigvals) if abs_neg_eigvals else 0
if adjustment:
    print('non PSD matrix: diagonal adjusment of {0}'.format(adjustment))

non PSD matrix: diagonal adjusment of 292.405301431


## Retrieve hardness index for axioms ##

In [9]:
import json
with open('hardness.json', 'r') as f:
    hardness = json.load(f)

num_hard = sum(hardness)
num_easy = n - num_hard
hardness_count = (num_easy, num_hard)

print 'There are {} easy and {} hard axioms'.format(num_easy, num_hard)

There are 1410 easy and 34 hard axioms


   ## Membership and possibility learning through repeated hold-out   

In [72]:
from possibilearn import *
from possibilearn.kernel import PrecomputedKernel
from possibilearn.fuzzifiers import LinearFuzzifier

import csv
import matplotlib.pyplot as plt
import gc

def estimate_possibility(n_all, mu_all, k, g, cs, ks, num_holdouts, percentages, fuzzifier,
                         hardness_class,
                         verbose=False):
    assert(len(mu_all)==n_all)
    axiom_indices = [i for i in range(n_all) if hardness[i]==hardness_class]
    n = hardness_count[hardness_class]
    mu = [mu_all[i] for i in range(len(mu_all)) if hardness[i]==hardness_class]
    assert(len(axiom_indices)==len(mu)==n)

    paired_axioms = [axiom_indices[i:i+2] for i in range(0, n, 2)]
    paired_labels = [mu[i:i+2] for i in range(0, n, 2)]

    metrics_membership_rmse = []
    metrics_membership_median = []
    metrics_membership_stdev = []

    metrics_possibility_rmse = []
    metrics_possibility_median = []
    metrics_possibility_stdev = []

    for h in range(num_holdouts):
        (paired_values_train,
         paired_values_validate,
         paired_values_test,
         paired_mu_train,
         paired_mu_validate,
         paired_mu_test) = split_indices(paired_axioms, paired_labels, percentages)

        if verbose:
            print 'holdout {} of {}'.format(h, num_holdouts)
    
        best_c, _, result = model_selection_holdout(paired_values_train, paired_mu_train,
                                                    paired_values_validate, paired_mu_validate,
                                                    cs, ks,
                                                    sample_generator=g,
                                                    log=True,
                                                    adjustment=adjustment,
                                                    fuzzifier=fuzzifier,
                                                    verbose=verbose)
        if best_c is None:
            if verbose:
                print 'in holdout {} optimization always failed!'.format(h)
            continue
    
        if verbose:
            print 'in holdout {} best C is {}'.format(h, best_c)
        estimated_membership = result[0]
    
        # values and labels are still paired, we need to flatten them out
        values_test = flatten(paired_values_test)
        mu_test = flatten(paired_mu_test)

        membership_square_err = [(estimated_membership(v) - m)**2 for v, m in zip(values_test, mu_test)]
        membership_rmse = math.sqrt(sum(membership_square_err) / len(values_test))
        metrics_membership_rmse.append(membership_rmse)
    
        membership_median = np.median(membership_square_err)
        metrics_membership_median.append(membership_median)
    
        membership_stdev = np.std(membership_square_err)
        metrics_membership_stdev.append(membership_stdev)
    
        estimated_mu = map(estimated_membership, values_test)
        actual_possibility = [mfi - mnotfi for mfi, mnotfi in zip(mu_test[::2], mu_test[1::2])]
        estimated_possibility = [mfi - mnotfi
                                 for mfi, mnotfi in zip(estimated_mu[::2], estimated_mu[1::2])]
    
        possibility_square_err = [(actual - estimated)**2
                              for actual, estimated in zip(actual_possibility, estimated_possibility)]
        possibility_rmse = math.sqrt(sum(possibility_square_err) / len(possibility_square_err))
        metrics_possibility_rmse.append(possibility_rmse)
    
        possibility_median = np.median(possibility_square_err)
        metrics_possibility_median.append(possibility_median)
    
        possibility_stdev = np.std(possibility_square_err)
        metrics_possibility_stdev.append(possibility_stdev)
    
        indices = ['-'.join(map(str, pair)) for pair in paired_values_test]

        results = [(i, phi, notphi, max(phi, notphi), ephi, enotphi, max(ephi, enotphi), p, ep, (p - ep)**2)
                   for i, phi, notphi, p, ephi, enotphi, ep in zip(indices, mu_test[::2], mu_test[1::2], actual_possibility,
                                                            estimated_mu[::2], estimated_mu[1::2], estimated_possibility)]

        results.sort(key = lambda r: r[-1])        
        errors = [r[-1] for r in results]

    if verbose:
        print 'Membership average values:'
        print 'RMSE: {}'.format(np.average(metrics_membership_rmse))
        print 'Median: {}'.format(np.average(metrics_membership_median))
        print 'STDEV: {}'.format(np.average(metrics_membership_stdev))

        print 'Possibility average values:'
        print 'RMSE: {}'.format(np.average(metrics_possibility_rmse))
        print 'Median: {}'.format(np.average(metrics_possibility_median))
        print 'STDEV: {}'.format(np.average(metrics_possibility_stdev))
    
    return (np.average(metrics_membership_rmse),
            np.average(metrics_membership_median),
            np.average(metrics_membership_stdev),
            np.average(metrics_possibility_rmse),
            np.average(metrics_possibility_median),
            np.average(metrics_possibility_stdev))


In [84]:
from possibilearn.fuzzifiers import *

hardness_class = 1

k = PrecomputedKernel(gram)
axiom_indices = axiom_indices = [i for i in range(n) if hardness[i]==hardness_class]

def g(m):
    return np.random.choice(axiom_indices, m if m <= len(axiom_indices) else len(axiom_indices))

cs = (0.07, 0.1, 0.3, 0.5, 0.7, 1, 10, 100)
cs = np.arange(0.05, 1, 0.01)
ks = (k,)

num_holdouts = 10
percentages = (.8, .1, .1)

fuzzifiers = [CrispFuzzifier(), LinearFuzzifier(), QuantileConstantPiecewiseFuzzifier(),
              QuantileLinearPiecewiseFuzzifier()] + [ExponentialFuzzifier(gamma) 
                                                     for gamma in np.arange(0.1, 1, 0.1)] + \
              [ExponentialFuzzifier(gamma) for gamma in np.arange(0.91, 1, 0.01)]

In [85]:
result = [estimate_possibility(n, mu, k, g,
                               cs, ks, num_holdouts,
                               percentages, f,
                               hardness_class,
                               verbose=False) for f in fuzzifiers]

In [86]:
import pandas as pd

pd.DataFrame(result, columns=('$\mu$ RMSE', '$\mu$ median', '$\mu$ STD',
                              '$\pi$ RMSE', '$\pi$ median', '$\pi$ STD'),
            index=[f.name for f in fuzzifiers])

,$\mu$ RMSE,$\mu$ median,$\mu$ STD,$\pi$ RMSE,$\pi$ median,$\pi$ STD
Crisp,0.916387,0.936732,0.217403,1.804482,3.432454,0.659469
Linear,0.783529,0.697548,0.378821,1.470733,1.991265,1.066544
QuantileConstPiecewise,0.781112,0.655617,0.309724,1.505775,2.493840,0.824312
QuantileLinPiecewise,0.759446,0.651334,0.377025,1.441806,2.029638,1.116787
Exponential(0.1),0.900859,0.915939,0.212241,1.771254,3.159936,0.525210
Exponential(0.2),0.849560,0.813026,0.295438,1.654272,3.069841,0.856402
Exponential(0.3),0.840431,0.776097,0.291393,1.636222,3.023952,0.848018
Exponential(0.4),0.750790,0.554311,0.291441,1.447816,2.160981,0.704323
Exponential(0.5),0.679367,0.417354,0.356223,1.223093,1.480594,0.488154
Exponential(0.6),0.668848,0.387990,0.363874,1.210975,1.552695,0.578769


In [ ]:
easy_axioms, hard_axioms split(axioms)

easy_axioms_train, easy_axioms_test = split(easy_axioms)
hard_axioms_train, hard_axioms_test = split(hard_axioms)

learn_easy_classifier(easy_axioms_train)
learn_hard_classifier(hard_axioms_train)

In [90]:
from possibilearn import split_indices, flatten

def learn_two_stage_classifier(n, mu, hardness,
                               easy_c, hard_c,
                               easy_sample_generator, hard_sample_generator,
                               easy_fuzzifier, hard_fuzzifier):
    assert(len(mu)==n)
    hard_axiom_indices = [i for i in range(n) if hardness[i]]
    num_hard_axioms = len(hard_axiom_indices)
    mu_hard_axioms = [mu[i] for i in range(n) if hardness[i]]
    
    easy_axiom_indices = [i for i in range(n) if not hardness[i]]
    num_easy_axioms = len(easy_axiom_indices)
    mu_easy_axioms = [mu[i] for i in range(n) if not hardness[i]]
    
    assert(num_easy_axioms + num_hard_axioms == n)

    paired_hard_axioms = [hard_axiom_indices[i:i+2] for i in range(0, num_hard_axioms, 2)]
    paired_hard_labels = [mu_hard_axioms[i:i+2] for i in range(0, num_hard_axioms, 2)]
    
    paired_easy_axioms = [easy_axiom_indices[i:i+2] for i in range(0, num_easy_axioms, 2)]
    paired_easy_labels = [mu_easy_axioms[i:i+2] for i in range(0, num_easy_axioms, 2)]
    
    percentages = (.9, 0, .1)

    (paired_easy_axioms_train,
     _,
     paired_easy_axioms_test,
     paired_easy_mu_train,
     _,
     paired_easy_mu_test) = split_indices(paired_easy_axioms, paired_easy_labels, percentages)
    
    (paired_hard_axioms_train,
     _,
     paired_hard_axioms_test,
     paired_hard_mu_train,
     _,
     paired_hard_mu_test) = split_indices(paired_hard_axioms, paired_hard_labels, percentages)
    
    easy_axioms_train = flatten(paired_easy_axioms_train)
    easy_axioms_test = flatten(paired_easy_axioms_test)
    easy_mu_train = flatten(paired_easy_mu_train)
    easy_mu_test = flatten(paired_easy_mu_test)
    
    hard_axioms_train = flatten(paired_hard_axioms_train)
    hard_axioms_test = flatten(paired_hard_axioms_test)
    hard_mu_train = flatten(paired_hard_mu_train)
    hard_mu_test = flatten(paired_hard_mu_test)
    
    easy_result = possibility_learn(easy_axioms_train, easy_mu_train, easy_c, k,
                                          sample_generator=easy_sample_generator,
                                          adjustment=adjustment,
                                          fuzzifier=easy_fuzzifier,
                                          crisp=False)
    hard_result = possibility_learn(hard_axioms_train, hard_mu_train, hard_c, k,
                                          sample_generator=hard_sample_generator,
                                          adjustment=adjustment,
                                          fuzzifier=hard_fuzzifier,
                                          crisp=False)
    return (easy_result, hard_result,
            easy_axioms_test, easy_mu_test, hard_axioms_test, hard_mu_test)

In [166]:
easy_c = 0.1
hard_c = 0.92

hard_axiom_indices = [i for i in range(n) if hardness[i]]
easy_axiom_indices = [i for i in range(n) if not hardness[i]]

def easy_sample_generator(m):
    return np.random.choice(easy_axiom_indices, m if m <= len(easy_axiom_indices) \
                            else len(easy_axiom_indices))

def hard_sample_generator(m):
    return np.random.choice(hard_axiom_indices, m if m <= len(hard_axiom_indices) \
                            else len(hard_axiom_indices))

def collect_pairs(l):
    return [l[i:i+2] for i in range(0, len(l), 2)]

def two_stage_experiment():
    easy_fuzzifier = CrispFuzzifier()
    hard_fuzzifier = ExponentialFuzzifier(0.92)

    easy_result, hard_result, \
    easy_axioms_test, easy_mu_test, \
    hard_axioms_test, hard_mu_test = learn_two_stage_classifier(n, mu, hardness,
                                                            easy_c, hard_c,
                                                            easy_sample_generator,
                                                            hard_sample_generator,
                                                            easy_fuzzifier, hard_fuzzifier)
    easy_membership = easy_result[0]
    hard_membership = hard_result[0]
    
    paired_easy_axioms_test = collect_pairs(easy_axioms_test)
    paired_easy_mu_test = collect_pairs(easy_mu_test)

    paired_hard_axioms_test = collect_pairs(hard_axioms_test)
    paired_hard_mu_test = collect_pairs(hard_mu_test)

    paired_axioms_test = paired_easy_axioms_test + paired_hard_axioms_test
    paired_mu_test = paired_easy_mu_test + paired_hard_mu_test
    
    def two_stage_classifier(axiom_pair):
        mus = map(easy_membership, axiom_pair)
        p = mus[0] - mus[1]
        if np.abs(p) < 0.7:
            mus = map(hard_membership, axiom_pair)
            p = mus[0] - mus[1]
        return p
    
    def get_performance(actual, estimated):
        actual_p = [mus[0] - mus[1] for mus in actual]
        estimated_p = map(two_stage_classifier, estimated)
        result = map(lambda p: (p[0]-p[1])**2, zip(actual_p, estimated_p))
        return (np.mean(result), np.median(result))
    
    return (get_performance(paired_mu_test, paired_axioms_test),
            get_performance(paired_easy_mu_test, paired_easy_axioms_test),
            get_performance(paired_hard_mu_test, paired_hard_axioms_test))

In [168]:
final_result = [two_stage_experiment() for _ in range(10)]



In [176]:
import pandas as pd

In [178]:
summary = pd.DataFrame(map(flatten, final_result),
             columns=('mean, all', 'median, all',
                      'mean, easy', 'median, easy',
                      'mean, hard', 'median, hard'))

In [180]:
summary.mean()

mean, all       0.370555
median, all     0.000957
mean, easy      0.276649
median, easy    0.000897
mean, hard      3.704236
median, hard    3.704236
dtype: float64